In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env

In [2]:
%load_ext autoreload
%autoreload 2

In [12]:
g_places_api_key = env.g_places_api_key

In [16]:
import json

# Define the URL and headers
url = 'https://places.googleapis.com/v1/places:searchText'
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': g_places_api_key,
    'X-Goog-FieldMask': 'places.displayName,places.formattedAddress'  # Specify the fields you want
}

# Define the query
data = {
    'textQuery': 'YUMMY JUICE BAR'
}


In [17]:

# Make the POST request
response = requests.post(url, headers=headers, json=data)

# Print the response
if response.ok:
    places = response.json()['places']
    for place in places:
        print(json.dumps(place, indent=2))
else:
    print(f'Error: {response.status_code}')
    print(response.text)


{
  "formattedAddress": "737 Lydig Ave, Bronx, NY 10462, USA",
  "displayName": {
    "text": "Yummy Juice Bar",
    "languageCode": "en"
  }
}


In [9]:
import requests
import json


# Formulate the request
# url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query=YUMMY+JUICE+BAR+737A+LYDIG+AVENUE+Bronx+NY&key={g_places_api_key}'


url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query=mexican&key={g_places_api_key}'

# Make the request
response = requests.get(url)

# Parse the response
places = response.json()['results']

# Print the first result (if any)
if places:
    print(json.dumps(places[0], indent=2))
else:
    print('No places found')


No places found


In [11]:
print(response.text)


{
   "error_message" : "This API project is not authorized to use this API.",
   "html_attributions" : [],
   "results" : [],
   "status" : "REQUEST_DENIED"
}


In [13]:
import requests
import json


# Formulate the request
# url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query=YUMMY+JUICE+BAR+737A+LYDIG+AVENUE+Bronx+NY&key={g_places_api_key}'


url = f'https://places.googleapis.com/v1/places:searchText/json?query=mexican&key={g_places_api_key}'

# Make the request
response = requests.get(url)

# Parse the response
places = response.json()['results']

# Print the first result (if any)
if places:
    print(json.dumps(places[0], indent=2))
else:
    print('No places found')


JSONDecodeError: Expecting value: line 1 column 1 (char 0)